In [ ]:
from IPython.display import clear_output

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown 
!unzip Celeb-DF-v2.zip
!pip install retina-face
clear_output()

In [ ]:
!pip install ffmpeg-python

In [ ]:
import os
import random
import csv
import ffmpeg
import subprocess
import cv2

from retinaface import RetinaFace
import numpy as np
import shutil
import matplotlib.image
import matplotlib.pyplot as plt
from numba import njit

In [ ]:
def deleteFolder(folder):
    lista = os.listdir(folder)
    for l in lista:
        l = os.path.join(folder, l)
        if os.path.isfile(l):
            os.remove(l)
        else:
            deleteFolder(os.path.join(folder, l))
    os.rmdir(folder)

deleteFolder("/content/Celeb-DF-V2-subclips")

In [ ]:
super = "Celeb-DF-V2-subclips"
sub = ["Images", "Videos"]
subsub = ["Real", "Fake"]

os.mkdir(super)
for s in sub:
    os.mkdir(os.path.join(super, s))
    for ss in subsub:
        os.mkdir(os.path.join(super, s, ss))

In [ ]:
def select_random_frames(video_path, n):
    # Get total number of frames in the video
    probe = ffmpeg.probe(video_path)
    total_frames = int(probe['streams'][0]['nb_frames'])

    # Generate a list of n unique random frame numbers
    if n > total_frames: return []
    random_frames = random.sample(range(1, total_frames + 1), n)
    random_frames.sort()  # Sort the frames in ascending order

    return random_frames

def save_frames_to_csv(video_path, frames, output_csv):
    # with open(output_csv, 'w', newline='') as csvfile:
    #     csv_writer = csv.writer(csvfile)
    #     csv_writer.writerow(['video_path'] + [f'frame_{i}' for i in range(1, len(frames) + 1)])
    #     csv_writer.writerow([video_path] + frames)
    file_exists = os.path.isfile(output_csv)

    with open(output_csv, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        if not file_exists:
            csv_writer.writerow(['video_path'] + [f'frame_{i}' for i in range(1, len(frames) + 1)])
        csv_writer.writerow([video_path] + frames)

def extract_frames(video_path, frames, output_folder):

    # Opening video
    cam = cv2.VideoCapture(video_path)
    original_video_name = "".join(os.path.basename(video_path).split(".")[:-1])

    new_frames_path = []

    #Extracting the frames
    current_frame = 0
    while True:
        ret, frame = cam.read()

        if not ret:
            break

        elif current_frame in frames:

            new_frame_path = os.path.join(output_folder, original_video_name + "--" +str(current_frame) + ".jpg")

            saved = cv2.imwrite(new_frame_path, frame)

            if not saved:
                print("video_path:",video_path,"- new_frame_path:", new_frame_path)

            new_frames_path.append(new_frame_path)

        current_frame += 1

    cam.release()
    if(current_frame == 0): print("Current_frame = 0 from video_path:", video_path)

    return new_frames_path

def generate_video_from_frames(image_paths, output_video_path, fps = 30):

    #Setting the output file name
    file_name = os.path.basename(image_paths[0]).split('.')[0].split("--")[0]
    file_name += ".mp4"
    output_video_path = os.path.join(output_video_path, file_name)

    # Get the dimensions of the first image
    first_image = cv2.imread(image_paths[0])
    height, width, _ = first_image.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Loop through each image and add it as a frame to the video
    for image_path in image_paths:
        frame = cv2.imread(image_path)
        out.write(frame)

    # Release VideoWriter and destroy any OpenCV windows
    out.release()
    cv2.destroyAllWindows()
    print(f"Video created: {output_video_path}")


In [ ]:
@njit
def begining_ending_center(target, original):
    begining = (target - original) // 2
    ending = begining + original
    return begining, ending

@njit
def paste_image_center(target_width, target_height, img_face):
    cropped_image_width, cropped_image_height  = img_face.shape[1], img_face.shape[0]

    #This array will represent a new image with black borders
    new_img = np.empty((target_height, target_width, 3), dtype=np.int32)
    new_img[:] = 0  # Initialize with zeros

    #Getting the pixels where the image will be placed to be in the center of the image with black border
    begining_image_width, ending_image_width   = begining_ending_center(target_width, cropped_image_width)
    begining_image_height, ending_image_height = begining_ending_center(target_height, cropped_image_height)

    #Placing the cropped face in the center of the image with black pixels
    for i in range(begining_image_height, ending_image_height):
        for j in range(begining_image_width, ending_image_width):
            new_img[i][j] = img_face[i - begining_image_height][j - begining_image_width]

    return new_img


def resize_image(img_face, target_width, target_height):
    return cv2.resize(img_face, dsize=(target_height, target_width), interpolation=cv2.INTER_CUBIC)


def begin_end(original_image_height, original_image_width, face_area):
    x1 = face_area[0]
    y1 = face_area[1]
    x2 = face_area[2]
    y3 = face_area[3]

    width = x2 - x1
    height = y3 - y1

    #Checking the begining and ending values of x and checking if they are not going to explode the image limits
    begin_x = x1 - int(width * 0.15)
    begin_x = begin_x if begin_x >= 0 else 0
    end_x = x2 + 1 + int(width * 0.15)
    end_x = end_x if end_x <= original_image_width else original_image_width

    #Checking the begining and ending values of y and checking if they are not going to explode the image limits
    begin_y = y1 - int(height * 0.15)
    begin_y = begin_y if begin_y >= 0 else 0
    end_y = y3 + 1 + int(height * 0.15)
    end_y = end_y if end_y <= original_image_height else original_image_height

    return begin_x, end_x, begin_y, end_y


def set_img_name(origin_path, destination_path):
    file_name = os.path.basename(origin_path)
    destination_path = os.path.join(destination_path, file_name)
    return destination_path


def cut_face_from_image(origin_path, destination_path_border):

    #Getting the position of the faces in the image
    faces = RetinaFace.detect_faces(img_path = origin_path)

    #REading the original image and getting its size information
    original_image = matplotlib.image.imread(origin_path)
    original_image_height = original_image.shape[0]
    original_image_width = original_image.shape[1]

    #If there is no face, just copy the image to the destination folder
    if not isinstance(faces, dict):
        #shutil.copy(origin_path, destination_path_border)
        return

    #Extracting the face from the original image and saving into one with a border
    face_area = faces["face_1"]["facial_area"]

    #Defining the begining and ending of the crop with an increase over the value returned from retinaface
    begin_x, end_x, begin_y, end_y = begin_end(original_image_height, original_image_width, face_area)

    #Cropping the image
    cropped_image_face = original_image[begin_y:end_y, begin_x:end_x]

    cropped_image_width, cropped_image_height  = cropped_image_face.shape[1], cropped_image_face.shape[0]
    target_width = 299
    target_height = 299

    #The image will be resized if the width or height are greater than the target. Otherwise it will add a black margin around it
    bordered_image = resize_image(cropped_image_face, target_width, target_height) if cropped_image_width > target_width or cropped_image_height > target_height else paste_image_center(target_width, target_height, cropped_image_face)
    image_destination = origin_path
    cv2.imwrite(image_destination, cv2.cvtColor(np.uint8(bordered_image), cv2.COLOR_RGB2BGR)) #CV2 saves image as BGR instead of RGB, so it is necessary to convert it to save correctly




In [ ]:
output_csv = '/content/Celeb-DF-V2-subclips/selected_frames_fake.csv'
output_folder = '/content/Celeb-DF-V2-subclips/Images/Fake'
output_video_path = '/content/Celeb-DF-V2-subclips/Videos/Fake'

diretorios_origem = ["/content/Celeb-synthesis"]

for diretorio in diretorios_origem:
    caminhos_arquivos = [os.path.join(diretorio, arq) for arq in os.listdir(diretorio)]

    caminhos_arquivos = caminhos_arquivos[4000:]

    tamanho = len(caminhos_arquivos)

    for i, video_path in enumerate(caminhos_arquivos):
        # Random frames
        random_frames = select_random_frames(video_path, 25)
        # The code will precede to save the images only if the random frames are correctly selected
        if random_frames != []:
            #Saving frames into the csv
            save_frames_to_csv(video_path, random_frames, output_csv)
            #Extracting the frames from the video
            files = extract_frames(video_path, random_frames, output_folder)
            #Extracting the faces from the images
            for f in files:
                cut_face_from_image(f, f)
            #Generating the video
            generate_video_from_frames([os.path.join(output_folder, f) for f in files], output_video_path)


        print(i,"/", tamanho)


Directory  /root /.deepface created
Directory  /root /.deepface/weights created
retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5
100%|██████████| 119M/119M [00:01<00:00, 88.4MB/s]


Video created: /content/Celeb-DF-V2-subclips/Videos/Fake/id46_id41_0009.mp4
0 / 1639
Video created: /content/Celeb-DF-V2-subclips/Videos/Fake/id10_id12_0005.mp4
1 / 1639
Video created: /content/Celeb-DF-V2-subclips/Videos/Fake/id20_id32_0007.mp4
2 / 1639
Video created: /content/Celeb-DF-V2-subclips/Videos/Fake/id35_id37_0002.mp4
3 / 1639
Video created: /content/Celeb-DF-V2-subclips/Videos/Fake/id53_id49_0004.mp4
4 / 1639
Video created: /content/Celeb-DF-V2-subclips/Videos/Fake/id9_id35_0002.mp4
5 / 1639
Video created: /content/Celeb-DF-V2-subclips/Videos/Fake/id30_id32_0007.mp4
6 / 1639
Video created: /content/Celeb-DF-V2-subclips/Videos/Fake/id31_id9_0006.mp4
7 / 1639
Video created: /content/Celeb-DF-V2-subclips/Videos/Fake/id17_id31_0005.mp4
8 / 1639
Video created: /content/Celeb-DF-V2-subclips/Videos/Fake/id31_id29_0006.mp4
9 / 1639
Video created: /content/Celeb-DF-V2-subclips/Videos/Fake/id48_id43_0005.mp4
10 / 1639
Video created: /content/Celeb-DF-V2-subclips/Videos/Fake/id59_id60

In [ ]:
!zip -r 'Celeb-DF-V2-subclips.zip' 'Celeb-DF-V2-subclips/'
clear_output()

In [ ]:
import shutil
shutil.move("/content/Celeb-DF-V2-subclips.zip", "/content/drive/MyDrive/Celeb-DF-V2-subclips-faces-Celeb-Synthesis-4000-fim.zip")

'/content/drive/MyDrive/Celeb-DF-V2-subclips-faces-Celeb-Synthesis-4000-fim.zip'